<a href="https://colab.research.google.com/github/SkShoheb33/Information-Retreval/blob/main/informationalRetreval1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import os
import re
import json
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
from pandas.core.internals import blocks
class InfoRetrival:
  def __init__(self,corpus):  
    self.corpus = corpus
    self.documents = []
    self.titles = []
    self.tokens = []
    self.termDocs = {}
    self.invertIndex = {}
    self.boolIndex = {}
    self.positionIndex = {}
    self.biwords = {}
    self.blocks = {}
    self.BSBI = {}
    self.SPIMI = {}
    for i in os.listdir(corpus):
      if i.split('.')[-1] == 'txt':
        self.documents.append(open(corpus+i).read())
        self.titles.append(i[:-4])
        # print(self.titles[-1])

  # def removeDuplicates(self,l)


  def makeTerms(self,document):
    terms = document.split(" ")
    n = len(terms)
    for i in range(n):
      if not terms[i].islower():
        terms[i] = terms[i].lower()
      terms[i] = re.sub(r'[^a-zA-Z0-9]', '', terms[i])
    return terms



  def removeStopWords(self,terms):
    # "i","am","it","is","the","we","was","a","an","and","or","to","so",
    stopWords = [".",";","!","@","#","$","%","^","&","*","(",")","{","}","[","]",":","'","/","<",">",'"',"+"]
    i = 0
    while i < len(terms):
      if terms[i] in stopWords  or len(terms[i])==0:
        terms.remove(terms[i])
      else:
        i+=1
    return terms


  def stemming(self):
    ps = PorterStemmer()
    for title in self.termDocs:
      n = len(self.termDocs[title])
      for i in range(n):
        self.termDocs[title][i] = ps.stem(self.termDocs[title][i])

  def termsDocuments(self):
    for title,docs in zip(self.titles,self.documents):
      self.termDocs[title] = self.removeStopWords(self.makeTerms(docs))



  def tokenize(self):
    self.termsDocuments()
    self.stemming()
    for title in self.termDocs:
      n = len(self.termDocs[title])
      for i in range(n):
        if self.termDocs[title][i] in self.tokens:
          continue
        self.tokens.append(self.termDocs[title][i])
    self.tokens.sort()




  def invertedIndex(self):
    self.tokenize()
    for token in self.tokens:
      for doc in self.termDocs:
        if token in self.termDocs[doc]:
          if token not in self.invertIndex:
            self.invertIndex[token] = []
          self.invertIndex[token].append(doc)




  def booleanIndex(self):
    self.tokenize()
    for i in self.tokens:
      for j in self.titles:
        if i not in self.boolIndex:
          self.boolIndex[i] = []
        if j in self.invertIndex[i]:
          self.boolIndex[i].append(1)
        else:
          self.boolIndex[i].append(0)


          
  def positionalIndex(self):
    ps = PorterStemmer()
    for i in self.tokens:
      self.positionIndex[i] = {}
      for j in self.titles:
        document = open(self.corpus+(j+".txt")).read().lower().split(" ")
        n = len(document)
        row = []
        for k in range(n):
          if i == ps.stem(document[k]):
            row.append(k)
        if len(row)>0:
          self.positionIndex[i][j] = row



  def biwordIndexing(self):
    for token in self.tokens:
      n = len(token)
      if n>0:
        l = ['$'+token[0]]
        if ('$'+token[0]) not in self.biwords :
          self.biwords['$'+token[0]] = [token]
        else:
          self.biwords['$'+token[0]].append(token)
        for i in range(n-1):
          if (token[i:i+2]) not in self.biwords :
            self.biwords[token[i:i+2]] = [token]
          else:
            self.biwords[token[i:i+2]].append(token)
        if (token[n-1]+'$') not in self.biwords :
          self.biwords[token[n-1]+'$'] = [token]
        else:
          self.biwords[token[n-1]+'$'].append(token)

  def wildcardQuery(self,query):
    s = ['$'+query[0]]
    k = 0
    if s[-1] in ["*$","$*"]:
      s = ['$'+query[1]]
      k = 1
    for i in range(k,len(query)-1):
      if '*' not in query[i:i+2]:
        s.append(query[i:i+2])
    s.append(query[-1]+'$')
    result = set(self.biwords[s[0]])
    for i in range(1,len(s)):
      result = result.intersection(set(self.biwords[s[i]]))
    return list(result)


  def printBooleanIndex(self):
    return pd.DataFrame(self.boolIndex,index=self.titles)




  def boolean_and(self,row1,row2):
    n = len(row1)
    for i in range(n):
      if row1[i] == row2[i] == 1:
        row1[i] = 1
      else:
        row1[i] = 0
    return row1



  def boolean_or(self,row1,row2):
    n = len(row1)
    for i in range(n):
      if row1[i] == 1 or row2[i] == 1:
        row1[i] = 1
      else:
        row1[i] = 0
    return row1



  def boolean_not(self,row):
    n = len(row)
    for i in range(n):
      if row[i] :
        row[i] = 0
      else:
        row[i] = 1
    return row



  def booleanQuery(self,query):
    query = query.split(" ")
    opearator = ""
    flag = False
    n = 10
    result = []
    for i in query:
      if i in ['and','or']:
        opearator = i
      elif i == 'not':
        flag = True
      else:
        if flag :
          flag = False
          result = self.boolean_not(self.boolIndex[i])
        if opearator == "and":
          result = self.boolean_and(result,self.boolIndex[i])
          operator = ""
        elif opearator == "or":
          result = self.boolean_or(result,self.boolIndex[i])
          operator = ""
        else:
          result = self.boolIndex[i]
    ans = []
    for i in range(n):
      if result[i]:
        ans.append(self.titles[i])
    return ans


  def commonDocument(self,terms):
    commonDocs = set(self.positionIndex[terms[0]])
    for i in range(1,len(terms)):
      commonDocs = commonDocs.intersection(self.positionIndex[terms[i]])
    return list(commonDocs)


  def phraseQuery(self,query):
    print("HELLo")
    positions = {}
    ps = PorterStemmer()
    query = query.lower().split(" ")
    n = len(query)
    for i in range(n):
      query[i] = ps.stem(query[i])
    result = []
    commonDocs = self.commonDocument(query)
    for doc in commonDocs:
      for position in self.positionIndex[query[0]][doc]:
        i = 1
        while i<n:
          if (position+i) in self.positionIndex[query[i]][doc]:
            i+=1
          else:
            break
        if i == n:
          result.append(doc)
    return result


  def correctWords(self,word):
    result = {}
    row = list(word)
    for token in self.tokens:
      dp = []
      column = list(token)
      for i in range(len(row)+1):
        dp.append([0 for j in range(len(column)+1)])
      for i in range(len(row)+1):
        for j in range(len(column)+1):
          if j == 0:
            dp[i][j] = i
          elif i == 0:
            dp[i][j] = j
      for i in range(1,len(row)+1):
        for j in range(1,len(column)+1):
          if row[i-1] == column[j-1]:
            dp[i][j] = min(dp[i-1][j],dp[i][j-1],dp[i-1][j-1])
          else:
            dp[i][j] = min(dp[i-1][j],dp[i][j-1],dp[i-1][j-1])+1
      # if dp[-1][-1] in [1,2]:
      if dp[-1][-1] not in result:
        result[dp[-1][-1]] = []
      result[dp[-1][-1]].append(token)
      # print(dp[-1][-1])
    return result[min(result.keys())]


  def leastFactor(self,n):
    for i in range(2,n):
      if n%i == 0:
        return i
    return 1            


  def makeBlock(self):
    ps = PorterStemmer()
    n = len(self.titles)
    b = self.leastFactor(n)
    c = 0
    self.blocks = {}
    c = 0
    # print(b,n)
    for i in range(0,n,b):
      self.blocks["block"+str(c+1)] = {}
      for j in range(i,i+b):
        terms = self.removeStopWords(self.makeTerms(open(self.corpus+self.titles[j]+".txt").read()))
        for term in terms:
          term = ps.stem(term)
          if term not in self.blocks["block"+str(c+1)]:
            self.blocks["block"+str(c+1)][term] = []
          if self.titles[j] not in self.blocks["block"+str(c+1)][term]:
            self.blocks["block"+str(c+1)][term].append(self.titles[j])
      c+=1



  def mergeDocs(self,doc1,doc2):
    doc1.sort()
    doc2.sort()
    m = len(doc1)
    n = len(doc2)
    result = []
    i,j = 0,0
    while i<m and j<n:
      if doc1[i] == doc2[j]:
        result.append(doc1[i])
        i+=1
        j+=1
      elif doc1[i] < doc2[j]:
        result.append(doc1[i])
        i+=1
      else:
        result.append(doc2[j])
        j+=1
    while i < m:
      result.append(doc1[i])
      i+=1
    while j < n:
      result.append(doc2[j])
      j+=1
    return result



  def mergeBlocks(self,blockA,blockB):
    termsA = list(blockA.keys())
    termsB = list(blockB.keys())
    termsA.sort()
    termsB.sort()
    m = len(termsA)
    n = len(termsB)
    result = {}
    i,j = 0,0
    while i < m  and j <n :
      if termsA[i] == termsB[j]:
        result[termsA[i]] = self.mergeDocs(blockA[termsA[i]],blockB[termsB[j]])
        i+=1
        j+=1
      elif termsA[i] < termsB[j]:
        result[termsA[i]] =blockA[termsA[i]]
        i+=1
      else:
        result[termsB[j]] = blockB[termsB[j]]
        j+=1
    while  i < m:
      result[termsA[i]] = blockA[termsA[i]]
      i+=1
    while  j < n:
      result[termsB[j]] = blockB[termsB[j]]
      j+=1
    return result


  def blockSortBasedIndexing(self):
    n = len(self.blocks)
    self.BSBI = self.mergeBlocks(self.blocks['block1'],{})
    for i in range(2,n+1):
      self.BSBI = self.mergeBlocks(self.BSBI,self.blocks[f'block{i}'])


  def singlePassInMemoryIndexing(self,corpus):
    path = self.corpus+'/dictionary.json'
    if os.path.exists(path):
      with open(path,'r') as f:
        self.SPIMI = json.load(f)
    for title in self.titles:
      if len(self.SPIMI)==0:
        for word in sorted(list(set(self.termDocs[title]))):
          self.SPIMI[word] = [title]
      else:
        for word in sorted(list(set(self.termDocs[title]))):
          if word in self.SPIMI:
            self.SPIMI[word].append(title)
          else:
            self.SPIMI[word] = [title]
    with open(path,'w') as f:
      json.dump(self.SPIMI,f)
    # return self.SPIMI

      
      


In [69]:
obj = InfoRetrival("/content/drive/MyDrive/stories/")
obj.invertedIndex()
obj.invertIndex
obj.booleanIndex()
obj.boolIndex
obj.printBooleanIndex()
obj.booleanQuery("tiger or tiger")
# obj.positionalIndex()
# obj.positionIndex
# obj.phraseQuery("li")
obj.biwordIndexing()
obj.biwords
obj.wildcardQuery("e*e")
obj.correctWords('lon')
obj.makeBlock()
obj.blocks
obj.blockSortBasedIndexing()
obj.BSBI
obj.singlePassInMemoryIndexing("/content/drive/MyDrive/stories")

In [ ]:
# obj.printBooleanIndex()

In [70]:
os.listdir('/content/drive/MyDrive/stories')
# sorted(obj.termDocs['doc1'])
# obj.SPIMI

['doc1.txt',
 'doc2.txt',
 'doc3.txt',
 'doc4.txt',
 'doc7.txt',
 'doc8.txt',
 'doc9.txt',
 'doc10.txt',
 'doc5.txt',
 'doc6.txt',
 'dictionary.json']